# Homework 7: Data Exploration and Analysis in SQL

This is an **individual** assignment. Total: 20 points. Due:**<span style="color:red">  Sunday, October 29, 10:00 pm </span>**.

**Objective**: The primary purpose of this assignment, titled "Data Exploration and Analysis in SQL: Insights from an Online Store's Transactions," is to provide students with hands-on experience in applying SQL queries for real-world data analysis. 

Consider the table `transactions` with the following schema:

```sql
    id INT AUTO_INCREMENT PRIMARY KEY,
    product_id INT,
    customer_id INT,
    quantity INT,
    price_per_unit DECIMAL(10, 2),
    purchase_date DATE
```

#### Task 1. Create the Table.

```sql
CREATE TABLE transactions (
	id INT AUTO_INCREMENT PRIMARY KEY,
	product_id INT,
    customer_id INT,
    quantity INT,
    price_per_unit DECIMAL(10, 2),
    purchase_date DATE
    );
```

#### Task 2. Use a Python script to populate the table with 10,000 records.

In [19]:
import mysql.connector
import random
from faker import Faker
from mysql.connector import Error
from datetime import datetime

fake = Faker()

try:
    # Establish a database connection
    connection = mysql.connector.connect(user='root', password='rootpassword', host='localhost', database='josh')

    cursor = connection.cursor()

    # SQL query for inserting data
    add_transaction = ("INSERT INTO transactions"
                "(product_id, customer_id, quantity, price_per_unit, purchase_date) "
                "VALUES (%s, %s, %s, %s, %s)")

    # Insert 10000 records
    for _ in range(10000):
        # Random data for each book
        product_id = random.randint(1,30)
        customer_id = random.randint(1,1000)
        quantity = random.randint(1,100)
        price_per_unit = random.uniform(10, 2)
        purchase_date = fake.date_between()
        
        # Data for insertion
        transaction_data = (product_id, customer_id, quantity, price_per_unit, purchase_date)

        # Insert new book
        cursor.execute(add_transaction, transaction_data)

    # Commit the transaction
    connection.commit()

except mysql.connector.Error as error:
    print(f"Failed to insert record into MySQL table {error}")

finally:
    # Close communication with the database
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

MySQL connection is closed


#### Task 3. List all unique products that have ever been sold.

```sql
SELECT COUNT(DISTINCT product_id) AS unique_products 
FROM transactions;
```
30 unique products

#### Task 4. Determine the average number of items sold per transaction.

```sql
SELECT AVG(quantity) AS avg_q FROM transactions;
```
50 items sold

#### Task 5. Analyze the total sales per year to observe annual sales trends.

```sql
SELECT YEAR(purchase_date) AS year_purch, SUM(quantity*price_per_unit) AS total_s FROM transactions
GROUP BY year_purch;
```
It rose quickly from 1993 to 2007 then plateaued

#### Task 6. Identify products that have been sold more than 50 times.

```sql
SELECT product_id, COUNT(*) AS prod_time FROM transactions
GROUP BY product_id HAVING COUNT(*) > 50;
```

#### Task 7. Calculate the total revenue generated from all transactions.

```sql
SELECT SUM(price_per_unit*quantity) AS total_p FROM transactions;
```
6008678.53

#### Task 8. Find the average value of a purchase across all transactions.

```sql
SELECT AVG(quantity*price_per_unit) AS avg_c FROM transactions;
```
300.433927

#### Task 9. Identify which product has been sold the most in terms of quantity.

```sql
SELECT product_id, SUM(quantity) AS prod_total FROM transactions
GROUP BY product_id ORDER BY prod_total DESC;
```
product 19, 37789

#### Task 10. Calculate the total sales amount each customer has contributed to.

```sql
SELECT customer_id, SUM(quantity*price_per_unit) AS total_s FROM transactions
GROUP BY customer_id ORDER BY total_s DESC;
```

#### Task 11. Summarize total sales revenue, broken down by each month.

```sql
SELECT
		CASE MONTH(purchase_date)
    WHEN 1 THEN 'January'
    WHEN 2 THEN 'February'
    WHEN 3 THEN 'March'
    WHEN 4 THEN 'April'
    WHEN 5 THEN 'May'
    WHEN 6 THEN 'June'
    WHEN 7 THEN 'July'
    WHEN 8 THEN 'August'
    WHEN 9 THEN 'September'
    WHEN 10 THEN 'October'
    WHEN 11 THEN 'November'
    WHEN 12 THEN 'December'
    END as month_n, SUM(quantity*price_per_unit) AS total_s FROM transactions
    GROUP BY month_n ORDER BY total_s DESC;
```
'October','535007.71'
'March','519346.39'
'July','518960.05'
'January','513601.98'
'May','511880.03'
'November','500839.90'
'December','497849.49'
'September','496765.89'
'June','485184.34'
'April','482952.24'
'August','477813.03'
'February','468477.48'

#### Task 12. Retrieve details of all transactions that occurred in the month of January.

```sql
SELECT * FROM transactions WHERE MONTH(purchase_date) = 1;
```

#### Task 13. Calculate the average number of transactions per month.

```sql
SELECT MONTH(purchase_date), COUNT(*) AS month_p FROM transactions
GROUP BY month_p;
```
'1','1721'
'2','1577'
'3','1741'
'4','1618'
'5','1692'
'6','1602'
'7','1715'
'8','1574'
'9','1644'
'10','1753'
'11','1646'
'12','1717'

#### Task 14.  List the top three transactions with the highest price per unit.

```sql
SELECT * FROM transactions
ORDER BY price_per_unit DESC LIMIT 3;
```
'13908','6','988','66','10.00','2003-05-17'
'12409','8','116','39','10.00','2017-03-21'
'12901','27','114','53','10.00','2013-05-15'

#### Task 15. Find customers whose total spending is above the average spending of all customers.

```sql
SELECT customer_id, SUM(quantity*price_per_unit) AS total_s FROM transactions
GROUP BY customer_id HAVING total_s > (SELECT AVG(quantity*price_per_unit) FROM transactions);
```